# 1. Setup libraries for Machine Learning 

The first step is to import all the libaries that will be needed to parse and explore the data

In [1]:
import sys
import pandas as pd
import base64

# Add the facets overview python code to the python path
import sys
# generic_feature_statistics_generator is in the facets directory
sys.path.append('./facets/facets_overview/python')


from generic_feature_statistics_generator import GenericFeatureStatisticsGenerator
from IPython.core.display import display, HTML

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
import numpy as np              # linear algebra
import pandas as pd             # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # plotting
import seaborn as sns           # plotting
import tensorflow as tf         # machine learning

# ML libraries 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

# path to save the model for future training or tensorflow serving
model_path = 'models/temp/iris'

# define feature columns for input feature data 
# *** pull this from the dataset?
prediction_feature_columns = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']

# define loacl file for training data with features and labels
csv_file = 'datasets/iris.csv'

# tell jupyter notebook to diplay plotted data inline
%matplotlib inline 

# verify version of tensorflow, should be 1.4+ to use new Estimators feature
print('TENSORFLOW VERSION: ', tf.__version__)


('TENSORFLOW VERSION: ', '1.4.1')


# 2. Import Dataset from local CSV file

In [2]:
# get full dataset from local CSV file
df = pd.read_csv(csv_file)
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


# 3. Clean up the Dataset

In [3]:
# drop unnecessary 'Id' column from the dataset, doesnt help predict the label
df.drop('Id', inplace=True, axis=1)

In [4]:
# ALWAYS randomize sample
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.9,3.0,5.1,1.8,Iris-virginica
1,5.2,4.1,1.5,0.1,Iris-setosa
2,5.3,3.7,1.5,0.2,Iris-setosa
3,7.2,3.6,6.1,2.5,Iris-virginica
4,5.1,3.8,1.5,0.3,Iris-setosa


In [5]:
# label encode the string label: Species
# it should be a numeric value, instead of a string value
# to do: use one-hot encoder instead of label encoder
le = LabelEncoder()
df.Species = le.fit_transform(df.Species)

# show the cleaned dataset
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.9,3.0,5.1,1.8,2
1,5.2,4.1,1.5,0.1,0
2,5.3,3.7,1.5,0.2,0
3,7.2,3.6,6.1,2.5,2
4,5.1,3.8,1.5,0.3,0


In [11]:
# set just the label column to y for the train_test_split()
# *** need to shuffle the dataset first?
data_label = df.Species
data_label.head()

0    2
1    0
2    0
3    2
4    0
Name: Species, dtype: int64

In [6]:
# remove the species column so that you can scale the rest of the values
# the values are already stored in y variable
# *** convert this to X values here, for clarity?
data_features = df.drop('Species', inplace=False, axis=1)
data_features.head()


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.9,3.0,5.1,1.8
1,5.2,4.1,1.5,0.1
2,5.3,3.7,1.5,0.2
3,7.2,3.6,6.1,2.5
4,5.1,3.8,1.5,0.3


In [9]:
# scale data for better results
# so that columns with larger values dont get weighed more
# example: age change of 40 years (significant) vs income change of $40 (not significant)

sc = StandardScaler(copy=True, with_mean=True, with_std=True)
scaled_features = pd.DataFrame(sc.fit_transform(data_features), columns=[data_features.columns], index=data_features.index)

# show scaled X values 
scaled_features.head() 
# THIS IS A DATAFRAME

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,0.068662,-0.124958,0.762759,0.790591
1,-0.779513,2.420475,-1.284407,-1.444450
2,-0.658345,1.494863,-1.284407,-1.312977
3,1.643844,1.263460,1.331416,1.710902
4,-0.900681,1.726266,-1.284407,-1.181504


In [12]:
# show labels (y values in equasion)
data_label.head()
# THIS IS A SERIES


0    2
1    0
2    0
3    2
4    0
Name: Species, dtype: int64

In [13]:
# do a 70/30 split of the data into a train_x and train_y (70% for training) 
# and a test_x and test_y (30% for validation) 
# X is a DATAFRAME of scaled features
# y is a SERIES of labels
train_x, test_x, train_y, test_y = train_test_split(scaled_features, data_label, test_size=0.30, random_state=101)
 

In [14]:
test_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 33 to 66
Data columns (total 4 columns):
SepalLengthCm    45 non-null float64
SepalWidthCm     45 non-null float64
PetalLengthCm    45 non-null float64
PetalWidthCm     45 non-null float64
dtypes: float64(4)
memory usage: 1.8 KB


# 4. Create Feature Columns List for Classifier

In [15]:
# Feature columns describe how to use the input.
# you need to define the name and type for each column of your training data
my_feature_columns = []

# numeric coulmns
my_feature_columns.append(tf.feature_column.numeric_column(key='SepalLengthCm'));
my_feature_columns.append(tf.feature_column.numeric_column(key='SepalWidthCm'));
my_feature_columns.append(tf.feature_column.numeric_column(key='PetalLengthCm'));
my_feature_columns.append(tf.feature_column.numeric_column(key='PetalWidthCm'));   

# other column types
# hashed strings or categorical features
# example:
# state = tf.feature_column.categorical_column_with_hash_bucket('state',100) 
# pick the right size for your data

    

# 5. Create Classifier from Feature Column list

In [16]:
# Build 2 hidden layer DNN with 10, 10 units respectively
# uses feature column types defined in the previous step
classifier = tf.estimator.DNNClassifier(
    # feature columns defined in the prev step
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=3,
    # save the model to a folder when run, reuse the same way
    model_dir=model_path
)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x121a728d0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'models/temp/iris', '_save_summary_steps': 100}


In [17]:
# setup the train input function using the train_x and train_y datasource
train_input_fn = tf.estimator.inputs.pandas_input_fn(train_x, y=train_y, num_epochs=5000, shuffle=True)


# 6. Train Classifier

In [18]:
# this is where training actually occurs, this will be a long running process on large datasets
classifier.train(input_fn=train_input_fn)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models/temp/iris/model.ckpt-4102
INFO:tensorflow:Saving checkpoints for 4103 into models/temp/iris/model.ckpt.
INFO:tensorflow:loss = 5.604626, step = 4103
INFO:tensorflow:global_step/sec: 380.548
INFO:tensorflow:loss = 0.6619314, step = 4203 (0.268 sec)
INFO:tensorflow:global_step/sec: 376.393
INFO:tensorflow:loss = 0.15026246, step = 4303 (0.265 sec)
INFO:tensorflow:global_step/sec: 380.205
INFO:tensorflow:loss = 0.35306287, step = 4403 (0.263 sec)
INFO:tensorflow:global_step/sec: 376.59
INFO:tensorflow:loss = 0.29147115, step = 4503 (0.269 sec)
INFO:tensorflow:global_step/sec: 343.415
INFO:tensorflow:loss = 0.1299471, step = 4603 (0.287 sec)
INFO:tensorflow:global_step/sec: 325.166
INFO:tensorflow:loss = 0.14205196, step = 4703 (0.311 sec)
INFO:tensorflow:global_step/sec: 385.17
INFO:tensorflow:loss = 0.22692503, step = 4803 (0.255 sec)
INFO:tensorflow:global_step/sec: 337.486
INFO:tensorflow:loss 

# 7. Test classifier with the 30% split data

In [19]:
# validate the model using the x_test values to get a prediction for each one
# then you can compare the predicted values against the known values in test_y
# which will give you an accuracy score in percentage
test_input_fn = tf.estimator.inputs.pandas_input_fn(test_x, num_epochs=1, shuffle=False)


In [20]:
test_result = list(classifier.predict(input_fn=test_input_fn))

print(test_result)

INFO:tensorflow:Restoring parameters from models/temp/iris/model.ckpt-8204
[{'probabilities': array([2.8553191e-07, 1.4047559e-07, 9.9999952e-01], dtype=float32), 'logits': array([-7.725773 , -8.435093 ,  7.3431387], dtype=float32), 'classes': array(['2'], dtype=object), 'class_ids': array([2])}, {'probabilities': array([9.9999976e-01, 3.7455752e-08, 2.6093042e-07], dtype=float32), 'logits': array([  6.5559154, -10.544189 ,  -8.603096 ], dtype=float32), 'classes': array(['0'], dtype=object), 'class_ids': array([0])}, {'probabilities': array([9.9999952e-01, 4.2856321e-08, 4.8011344e-07], dtype=float32), 'logits': array([  6.7573376, -10.208074 ,  -7.7919054], dtype=float32), 'classes': array(['0'], dtype=object), 'class_ids': array([0])}, {'probabilities': array([9.9999797e-01, 1.2909488e-06, 7.0771358e-07], dtype=float32), 'logits': array([ 6.238935 , -7.3211956, -7.92229  ], dtype=float32), 'classes': array(['0'], dtype=object), 'class_ids': array([0])}, {'probabilities': array([9.999

In [21]:
# build an array with just the predicted species (label encoded values) 
# filtering out all other prediction response data
predicted_y = []
for each in test_result:
    predicted_y.append(each['class_ids'][0])

In [22]:
# compare the predicted scores on the test data against the actual test_y values
# with both a confusion matrix and an accuracy score
print('OVERALL ACCURACY:')
print(accuracy_score(test_y, predicted_y))

print('')

print('CONFUSION MATRIX:')
print(confusion_matrix(test_y, predicted_y))


OVERALL ACCURACY:
0.9333333333333333

CONFUSION MATRIX:
[[13  1  0]
 [ 0 11  1]
 [ 0  1 18]]


# 8. Predict with new data records 

In [27]:
# test using single records (real world values)
# one Iris-setosa, one Iris-versicolor
new_records = pd.DataFrame([
    [5.1, 3.5, 1.4, 0.2],  # first value to predict
    [5.7, 2.8, 4.1, 1.3]], # second value to predict
    columns=prediction_feature_columns)
 
# these values must be scaled, to match what is used in the model
scaled_new_records = pd.DataFrame(sc.transform(new_records), columns=[new_records.columns], index=new_records.index)

#SepalLengthCm, SepalWidthCm, PetalLengthCm, PetalWidthCm
#-1.870024, -0.124958, -1.511870, -1.444450 : 0
# 1.522676, -0.124958, 1.217684, 1.185010 : 1


In [28]:
# predict using the model to get a prediction for each item passed
prediction_input_fn = tf.estimator.inputs.pandas_input_fn(scaled_new_records, num_epochs=1, shuffle=False)

In [29]:
prediction_result = list(classifier.predict(input_fn=prediction_input_fn))

prediction_result

INFO:tensorflow:Restoring parameters from models/temp/iris/model.ckpt-8204


[{'class_ids': array([0]),
  'classes': array(['0'], dtype=object),
  'logits': array([  6.447382, -11.199646,  -9.589509], dtype=float32),
  'probabilities': array([9.9999988e-01, 2.1676696e-08, 1.0845917e-07], dtype=float32)},
 {'class_ids': array([1]),
  'classes': array(['1'], dtype=object),
  'logits': array([-6.894826,  7.952074, -6.275165], dtype=float32),
  'probabilities': array([3.565109e-07, 9.999989e-01, 6.625030e-07], dtype=float32)}]

# 9. Decode new predictions

In [30]:
# for each of the results
i = 0
for each in prediction_result:
    # print the values used in the prediction
    print(new_records.iloc[[i]])
    
    # print the transformed label-encoded value
    print('*** PREDICTION: ', each['class_ids'][0], '(Label Decoded: ', le.inverse_transform(each['class_ids'][0]), ')')
    print('')
    
    # go to next row
    i = i + 1
    
#-1.870024, -0.124958, -1.511870, -1.444450 : 0
# 1.522676, -0.124958, 1.217684, 1.185010 : 1

   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
0            5.1           3.5            1.4           0.2
('*** PREDICTION: ', 0, '(Label Decoded: ', 'Iris-setosa', ')')

   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
1            5.7           2.8            4.1           1.3
('*** PREDICTION: ', 1, '(Label Decoded: ', 'Iris-versicolor', ')')



/Users/jameswolf/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jameswolf/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# 10. Save Model for Tensorflow Serving

In [31]:
export_dir="serving/models/iris/"

feature_spec = tf.feature_column.make_parse_example_spec(my_feature_columns)

input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

classifier.export_savedmodel(export_dir, input_receiver_fn, as_text=False) 

INFO:tensorflow:Restoring parameters from models/temp/iris/model.ckpt-8204
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: serving/models/iris/temp-1523681786/saved_model.pb


'serving/models/iris/1523681786'

# 11. Make predictions from the saved DNNClassifier model

https://www.kaggle.com/jeffcarp/example-save-and-load-a-tensorflow-model

In [32]:
# this should load the model from the last saved in the model_dir
# so everything it needs should be included below

# input column names
columns_names = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']

# where the model was last saved 
model_path = 'models/temp/iris' 

# define columns in model using the column_names list
hardcoded_feature_columns = []
for key in columns_names:
    hardcoded_feature_columns.append(tf.feature_column.numeric_column(key=key))
    print(key)

# create a classifier using the defined columns and the pre-defined model
saved_estimator = tf.estimator.DNNClassifier(
    # feature column array of tf.feature_column types, defined in the prev step
    feature_columns=hardcoded_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=3,
    # use saved model from folder
    model_dir=model_path
)

# test using single records (2 real world values)
# one Iris-setosa, one Iris-versicolor
records_to_predict = pd.DataFrame([
    [5.1, 3.5, 1.4, 0.2], # first records features
    [5.7, 2.8, 4.1, 1.3]],# second record features
    columns=columns_names)# defined column names
 
# these values must be scaled, to match what is used in the model
scaled_records_to_predict = pd.DataFrame(sc.transform(records_to_predict), columns=[records_to_predict.columns], index=records_to_predict.index)

# predict using the model to get a prediction for each item passed
saved_prediction_input_fn = tf.estimator.inputs.pandas_input_fn(scaled_records_to_predict, num_epochs=1, shuffle=False)

# make predictions and store as a list
saved_prediction_result = list(saved_estimator.predict(input_fn=saved_prediction_input_fn))

# output prediction values
saved_prediction_result

#SepalLengthCm, SepalWidthCm, PetalLengthCm, PetalWidthCm
#-1.870024, -0.124958, -1.511870, -1.444450 : 0
# 1.522676, -0.124958, 1.217684, 1.185010 : 1
#estimator_from_file = load_estimator_model('models/iris/1520662093')

SepalLengthCm
SepalWidthCm
PetalLengthCm
PetalWidthCm
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x123b40ed0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'models/temp/iris', '_save_summary_steps': 100}
INFO:tensorflow:Restoring parameters from models/temp/iris/model.ckpt-8204


[{'class_ids': array([0]),
  'classes': array(['0'], dtype=object),
  'logits': array([  6.447382, -11.199646,  -9.589509], dtype=float32),
  'probabilities': array([9.9999988e-01, 2.1676696e-08, 1.0845917e-07], dtype=float32)},
 {'class_ids': array([1]),
  'classes': array(['1'], dtype=object),
  'logits': array([-6.894826,  7.952074, -6.275165], dtype=float32),
  'probabilities': array([3.565109e-07, 9.999989e-01, 6.625030e-07], dtype=float32)}]

# END

Juest testing features and note below this line, not part of main execution

In [34]:
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.9,3.0,5.1,1.8,2
1,5.2,4.1,1.5,0.1,0
2,5.3,3.7,1.5,0.2,0
3,7.2,3.6,6.1,2.5,2
4,5.1,3.8,1.5,0.3,0


In [35]:
jsonstr = df.to_json(orient='records')

# Display the Dive visualization for this data
from IPython.core.display import display, HTML

HTML_TEMPLATE = """<link rel="import" href="/nbextensions/facets-dist/facets-jupyter.html">
        <facets-dive sprite-image-width="32" sprite-image-height="32" id="elem" height="600"></facets-dive>
        <script>
          var data = {jsonstr};
          document.querySelector("#elem").data = data;
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=jsonstr)
display(HTML(html))

In [40]:
# Add the facets overview python code to the python path
import sys
# generic_feature_statistics_generator is in the facets directory
sys.path.append('./facets/facets_overview/python')

# Calculate the feature statistics proto from the datasets and stringify it for use in facets overview
from generic_feature_statistics_generator import GenericFeatureStatisticsGenerator
import base64

gfsg = GenericFeatureStatisticsGenerator()
proto = gfsg.ProtoFromDataFrames([{'name': 'train', 'table': df}])
protostr = base64.b64encode(proto.SerializeToString()).decode("utf-8")






# Add the facets overview python code to the python path
import sys
sys.path.append('./python')

# Calculate the feature statistics proto from the datasets and stringify it for use in facets overview
from generic_feature_statistics_generator import GenericFeatureStatisticsGenerator
import base64

gfsg = GenericFeatureStatisticsGenerator()
proto = gfsg.ProtoFromDataFrames([{'name': 'train', 'table': cleanclicks}])
protostr = base64.b64encode(proto.SerializeToString()).decode("utf-8")

In [41]:
# Display the facets overview visualization for this data
from IPython.core.display import display, HTML

HTML_TEMPLATE = """<link rel="import" href="/nbextensions/facets-dist/facets-jupyter.html" >
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
html = HTML_TEMPLATE.format(protostr=protostr)
display(HTML(html))